In [ ]:
import os
import keras 
import tweepy
import joblib

import numpy as np 

ALLOWED_CHARS = [' ', '!', '?', ',', '.', '\'', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']

def clean_text(text):
    text = text.replace("@need_me_some_hp", "").lower()
    return  "".join([char for char in text if char in ALLOWED_CHARS])

def generate_new_text(seed, diversity, length=140):

    def sample(preds, temperature=1.0):
        # helper function to sample an index from a probability array
        preds = np.asarray(preds).astype('float64')
        preds = np.log(preds) / temperature
        exp_preds = np.exp(preds)
        preds = exp_preds / np.sum(exp_preds)
        probas = np.random.multinomial(1, preds, 1)
        return np.argmax(probas)
    
    seed = clean_text(seed)

    # pad or trim seed-sentence to 100 chars 
    # -> this is what the model was trained on
    if len(seed) > 100:
        generated = seed[-100:]
    elif len(seed) < 100:
        generated = (100 - len(seed)) * " " + seed

    sequence = np.asarray([char_to_idx[char] for char in generated])
    
    for i in range(length-len(seed)):
        preds = model.predict(sequence.reshape(-1,100), verbose=0)[0][-1]
        next_char_index = sample(preds, diversity)
        next_char = idx_to_char[next_char_index]

        generated += next_char
        sequence = np.append(sequence[1:], next_char_index)
    
    return generated.strip()[-length:]

# load model 
model = keras.models.load_model("./models/char_model_demo")
char_to_idx = joblib.load("./models/char_to_idx_demo")
idx_to_char = {idx:char for char, idx in char_to_idx.items()}

In [ ]:
text = ""
gen = generate_new_text(text, 0.4, 500)

print(gen)